In [227]:
# import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns

# from category_encoders import OneHotEncoder, MEstimateEncoder, GLMMEncoder, OrdinalEncoder, CatBoostEncoder
# from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedKFold, KFold
# from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, GradientBoostingRegressor
# from sklearn.ensemble import HistGradientBoostingRegressor, VotingRegressor, StackingRegressor, AdaBoostRegressor
# from sklearn.linear_model import Ridge, Lasso
# from sklearn.linear_model import QuantileRegressor
# from sklearn.linear_model import HuberRegressor
# from sklearn.cross_decomposition import PLSRegression
# from sklearn.neural_network import MLPRegressor
# from sklearn.metrics import median_absolute_error, roc_auc_score, roc_curve
# from sklearn.metrics.pairwise import euclidean_distances
# from sklearn.pipeline import Pipeline, make_pipeline
# from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
# from sklearn.preprocessing import FunctionTransformer, StandardScaler, MinMaxScaler, PowerTransformer
# from sklearn.compose import ColumnTransformer
# from sklearn.calibration import CalibratedClassifierCV
# from scipy.cluster.hierarchy import dendrogram, linkage
# from scipy.spatial.distance import squareform
# from xgboost import XGBRegressor
# from lightgbm import LGBMRegressor
# from catboost import CatBoostRegressor
from sklego.linear_model import LADRegression

import mlflow
import mlflow.sklearn

In [228]:
exp_name = "BaseModel"
if not mlflow.get_experiment_by_name(name=exp_name):
    mlflow.create_experiment(name=exp_name)
my_exp = mlflow.set_experiment(experiment_name=exp_name)

In [229]:
run_name = "LADRegression"
model = LADRegression()
active_run = mlflow.start_run(run_name=run_name)

In [230]:
from sklearn.model_selection import train_test_split

In [231]:
df = pd.read_csv("../data/train.csv")
df.drop(columns=["id"], inplace=True)
df = df.round(decimals=2)

In [232]:
X = df.drop(columns=["Hardness"])
y = df["Hardness"]

In [233]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=1)

In [234]:
model.fit(X_train, y_train)

LADRegression()

In [235]:
result = model.predict(X_test)

In [236]:
from sklearn.metrics import mean_squared_error

In [237]:
mse = mean_squared_error(result, y_test)

In [238]:
mlflow.set_experiment_tag(key="model", value=run_name)

In [239]:
mlflow.log_params(model.get_params())

In [240]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import explained_variance_score

In [241]:
mse = mean_squared_error(result, y_test)
mae = mean_absolute_error(result, y_test)
r2 = r2_score(result, y_test)
msle = mean_squared_log_error(result, y_test)
evs = explained_variance_score(result, y_test)

ValueError: Mean Squared Logarithmic Error cannot be used when targets contain negative values.

In [ ]:
mlflow.log_metrics(
    {
        "mean_squared_error": mse, 
        "mean_absolute_error": mae, 
        "r2_score": r2, 
        "mean_squared_log_error": msle, 
        "explained_variance_score": evs
    }
)

In [ ]:
my_model = mlflow.sklearn.log_model(model, "model")

c:\GitHub\Regression-with-a-Mohs-Hardness-Dataset\.venv\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\GitHub\Regression-with-a-Mohs-Hardness-Dataset\.venv\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [ ]:
my_model.run_id

'ab7dba0a9d144d829097c93418d6a41d'

In [ ]:
mlflow.end_run()

In [ ]:
df_test = pd.read_csv("../data/test.csv")
id = df_test["id"]
df_test.drop(columns=["id"], inplace=True)
result_submission = model.predict(df_test)
df_submission = pd.DataFrame(data={'id': id ,'Hardness': result_submission})
df_submission.to_csv("../submission/" + run_name + "_submission.csv  ", index=False)